In [3]:
import pandas as pd

In [4]:
ratings_path = 'data/ratings.csv'
movies_path = 'data/movies.csv'

ratings = pd.read_csv("/home/ajitesh/Documents/ML PROJECTS/Movie-Recommendation-System/data/ml-latest-small/ratings.csv")
movies = pd.read_csv("/home/ajitesh/Documents/ML PROJECTS/Movie-Recommendation-System/data/ml-latest-small/movies.csv")

#Display the first few rows of each dataset
print(ratings.head())
print(movies.head())

#check for missing values
print(ratings.isnull().sum())
print(movies.isnull().sum())

display(ratings.describe())
display(movies.info())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
movieId    0
title      0
genres     0
dtype: int64


,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


None

Data preprocessing

In this step, we'll clean and prepare the data for building the recommendation system. This involves:

Understanding the Data:
ratings.csv: Contains user IDs, movie IDs, ratings, and timestamps.
movies.csv: Contains movie IDs, titles, and genres.

Tasks:
Handle missing values (if any).
Merge the ratings and movies datasets for easier analysis.
Extract useful features (e.g., genres, average ratings).

In [5]:
#Merge ratings and movies datasets
merged_data = pd.merge(ratings, movies, on = 'movieId')

#Display the first few rows of the merged dataset
print("merged data")
display(merged_data.head())

#Check for missing values in the merged dataset
print("missing values")
print(merged_data.isnull().sum())

#Calculate the average rating of each movie
average_rating = merged_data.groupby('title')['rating'].mean()

#Display the top 10 movies with the highest average rating
print("Top 10 movies with the highest average rating")
display(average_rating.head(10))

#count the number of ratings for each movie
ratings_count = merged_data.groupby('title')['rating'].count().sort_values(ascending = False)

#Display the top 10 most rated movies
display(ratings_count.head(10))

merged data


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


missing values
userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64
Top 10 movies with the highest average rating


title
'71 (2014)                                 4.000000
'Hellboy': The Seeds of Creation (2004)    4.000000
'Round Midnight (1986)                     3.500000
'Salem's Lot (2004)                        5.000000
'Til There Was You (1997)                  4.000000
'Tis the Season for Love (2015)            1.500000
'burbs, The (1989)                         3.176471
'night Mother (1986)                       3.000000
(500) Days of Summer (2009)                3.666667
*batteries not included (1987)             3.285714
Name: rating, dtype: float64

title
Forrest Gump (1994)                          329
Shawshank Redemption, The (1994)             317
Pulp Fiction (1994)                          307
Silence of the Lambs, The (1991)             279
Matrix, The (1999)                           278
Star Wars: Episode IV - A New Hope (1977)    251
Jurassic Park (1993)                         238
Braveheart (1995)                            237
Terminator 2: Judgment Day (1991)            224
Schindler's List (1993)                      220
Name: rating, dtype: int64

In [6]:
min_ratings = 50

#filter movies
popular_movies = merged_data.groupby("title").filter(lambda x: x['rating'].count() >= min_ratings)

movie_stats = popular_movies.groupby("title").agg(
    average_rating=("rating", "mean"),
    num_ratings=("rating", "count")
)

#sort movies by average rating
movie_stats = movie_stats.sort_values(by = 'average_rating', ascending = False)

display(movie_stats.head(10))


,average_rating,num_ratings
title,,
"Shawshank Redemption, The (1994)",4.429022,317
"Godfather, The (1972)",4.289062,192
Fight Club (1999),4.272936,218
Cool Hand Luke (1967),4.271930,57
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),4.268041,97
Rear Window (1954),4.261905,84
"Godfather: Part II, The (1974)",4.259690,129
"Departed, The (2006)",4.252336,107
Goodfellas (1990),4.250000,126


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Create a user-movie matrix
user_movie_matrix = merged_data.pivot_table(index="userId", columns="title", values="rating")

# Fill missing values with 0 (assume unrated movies have a rating of 0)
user_movie_matrix = user_movie_matrix.fillna(0)

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix)

# Convert the similarity matrix into a DataFrame for easier interpretation
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Display the similarity matrix for the first 5 users
print("User Similarity Matrix (first 5 users):")
display(user_similarity_df.head())

# Function to recommend movies for a given user
def recommend_movies(user_id, num_recommendations=5):
    # Find similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]  # Exclude the user themselves
    
    # Get movies rated by similar users
    similar_users_ratings = user_movie_matrix.loc[similar_users]
    
    # Calculate the average rating for each movie (weighted by similarity)
    weighted_ratings = similar_users_ratings.T.dot(user_similarity_df[user_id].loc[similar_users])
    weighted_ratings = weighted_ratings / user_similarity_df[user_id].loc[similar_users].sum()
    
    # Recommend movies the user hasn't rated yet
    user_ratings = user_movie_matrix.loc[user_id]
    recommendations = weighted_ratings[user_ratings.isna()].sort_values(ascending=False).head(num_recommendations)
    
    return recommendations

# Recommend movies for a specific user (e.g., userId = 1)
user_id = 1
print(f"Top 5 movie recommendations for User {user_id}:")
display(recommend_movies(user_id))

User Similarity Matrix (first 5 users):


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.027283,0.059720,0.194395,0.129080,0.128152,0.158744,0.136968,0.064263,0.016875,...,0.080554,0.164455,0.221486,0.070669,0.153625,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,1.000000,0.000000,0.003726,0.016614,0.025333,0.027585,0.027257,0.000000,0.067445,...,0.202671,0.016866,0.011997,0.000000,0.000000,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,1.000000,0.002251,0.005020,0.003936,0.000000,0.004941,0.000000,0.000000,...,0.005048,0.004892,0.024992,0.000000,0.010694,0.012993,0.019247,0.021128,0.000000,0.032119
4,0.194395,0.003726,0.002251,1.000000,0.128659,0.088491,0.115120,0.062969,0.011361,0.031163,...,0.085938,0.128273,0.307973,0.052985,0.084584,0.200395,0.131746,0.149858,0.032198,0.107683
5,0.129080,0.016614,0.005020,0.128659,1.000000,0.300349,0.108342,0.429075,0.000000,0.030611,...,0.068048,0.418747,0.110148,0.258773,0.148758,0.106435,0.152866,0.135535,0.261232,0.060792


Top 5 movie recommendations for User 1:


Series([], dtype: float64)

In [8]:
# Calculate cosine similarity between movies
movie_similarity = cosine_similarity(user_movie_matrix.T)  # Transpose to make movies the rows

# Convert the similarity matrix into a DataFrame for easier interpretation
movie_similarity_df = pd.DataFrame(movie_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

# Display the similarity matrix for the first 5 movies
print("Movie Similarity Matrix (first 5 movies):")
display(movie_similarity_df.head())

# Function to recommend movies based on a user's preferences
def recommend_similar_movies(user_id, num_recommendations=5):
    # Get the user's ratings
    user_ratings = user_movie_matrix.loc[user_id]
    
    # Find movies the user has rated
    rated_movies = user_ratings[user_ratings > 0].index
    
    # Calculate a score for each movie based on similarity to rated movies
    similar_movies = movie_similarity_df[rated_movies].mean(axis=1)
    
    # Exclude movies the user has already rated
    similar_movies = similar_movies[~similar_movies.index.isin(rated_movies)]
    
    # Sort and return the top recommendations
    recommendations = similar_movies.sort_values(ascending=False).head(num_recommendations)
    return recommendations

# Recommend movies for a specific user (e.g., userId = 1)
user_id = 1
print(f"Top 5 movie recommendations for User {user_id} based on Item-Based Collaborative Filtering:")
display(recommend_similar_movies(user_id))

Movie Similarity Matrix (first 5 movies):


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot (2004),0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You (1997),0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


Top 5 movie recommendations for User 1 based on Item-Based Collaborative Filtering:


title
Ferris Bueller's Day Off (1986)    0.345526
Die Hard (1988)                    0.325395
Fifth Element, The (1997)          0.322012
Breakfast Club, The (1985)         0.321997
Mars Attacks! (1996)               0.318733
dtype: float64

In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# Create a User-Movie matrix for the training set
train_user_movie_matrix = train_data.pivot_table(index="userId", columns="title", values="rating").fillna(0)

# Function to predict ratings for the test set
def predict_ratings(user_id, movie_title):
    if movie_title in movie_similarity_df.columns:
        # Get similar movies
        similar_movies = movie_similarity_df[movie_title]
        # Get the user's ratings for those movies
        user_ratings = train_user_movie_matrix.loc[user_id]
        # Calculate the predicted rating as a weighted average
        predicted_rating = (similar_movies * user_ratings).sum() / similar_movies.sum()
        return predicted_rating
    else:
        return 0  # If the movie is not in the training set

# Evaluate the system using RMSE
test_data["predicted_rating"] = test_data.apply(
    lambda row: predict_ratings(row["userId"], row["title"]), axis=1
)

# Calculate RMSE
rmse = sqrt(mean_squared_error(test_data["rating"], test_data["predicted_rating"]))
print(f"Root Mean Square Error (RMSE): {rmse}")

Root Mean Square Error (RMSE): 3.231876843881025


In [10]:
# Normalize ratings by subtracting the user's mean rating
user_movie_matrix_normalized = user_movie_matrix.sub(user_movie_matrix.mean(axis=1), axis=0).fillna(0)

In [11]:
min_ratings = 100  # Increase the threshold
popular_movies = merged_data.groupby("title").filter(lambda x: x['rating'].count() >= min_ratings)

In [13]:
pip install flask

Note: you may need to restart the kernel to use updated packages.
